In [2]:
import rasterio
import numpy as np
file_name = '/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Biarritz/Biarritz_EPSG32630_1.tif'
file_name = '/data/DIGITANIE/Biarritz/Biarritz_EPSG32630_1.tif'
with rasterio.open(file_name) as src:
    print(src.meta)
    band1 = src.read(1)
    print('Band1 has shape', band1.shape)
    height = band1.shape[0]
    width = band1.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lons= np.array(xs)
    lats = np.array(ys)
    print('lons shape', lons.shape)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 2048, 'height': 2048, 'count': 4, 'crs': CRS.from_epsg(32630), 'transform': Affine(0.4996690458118865, 0.0, 620734.67813581,
       0.0, -0.4996690458118865, 4816930.568377791)}
Band1 has shape (2048, 2048)
lons shape (2048, 2048)


In [3]:
lons = lons[:100, :100]
lats = lats[:100, :100]

In [4]:
import pyproj as proj
import utm

# setup your projections
crs_src = proj.Proj(init='epsg:32630') # assuming you're using WGS84 geographic
crs_dst = proj.Proj(init='epsg:4326 ') # use a locally appropriate projected CRS

# then cast your geographic coordinate pair to the projected system
input_lon, input_lat = proj.transform(crs_src, crs_dst, lons, lats)
input_lon, input_lat

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3854905/320663268.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  input_lon, inpu

(array([[-1.50669398, -1.5066878 , -1.50668162, ..., -1.5060947 ,
         -1.50608852, -1.50608235],
        [-1.50669409, -1.50668791, -1.50668173, ..., -1.50609481,
         -1.50608864, -1.50608246],
        [-1.5066942 , -1.50668802, -1.50668184, ..., -1.50609492,
         -1.50608875, -1.50608257],
        ...,
        [-1.50670473, -1.50669855, -1.50669237, ..., -1.50610546,
         -1.50609928, -1.5060931 ],
        [-1.50670484, -1.50669866, -1.50669248, ..., -1.50610557,
         -1.50609939, -1.50609321],
        [-1.50670495, -1.50669877, -1.50669259, ..., -1.50610568,
         -1.5060995 , -1.50609333]]),
 array([[43.49555068, 43.4955506 , 43.49555052, ..., 43.49554285,
         43.49554277, 43.49554269],
        [43.49554618, 43.4955461 , 43.49554602, ..., 43.49553835,
         43.49553827, 43.49553819],
        [43.49554168, 43.4955416 , 43.49554152, ..., 43.49553385,
         43.49553377, 43.49553369],
        ...,
        [43.49511441, 43.49511433, 43.49511425, ..., 4

In [5]:
utm.from_latlon(input_lat, input_lon)

(array([[620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        ...,
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648]]),
 array([[4816930.31872322, 4816930.31872322, 4816930.31872322, ...,
         4816930.31872322, 4816930.31872322, 4816930.31872322],
        [4816929.81905417, 4816929.81905417, 4816929.81905417, ...,
         4816929.81905417, 48

In [ ]:
np.multiply(np.cos(lats), np.cos(lons))
np.multiply(np.cos(lats), np.sin(lons))

In [1]:
import rasterio

def read_window_from_big_raster(window, path, raster_path):
    with rasterio.open(path) as image_file:
        with rasterio.open(raster_path) as raster_file:
            left, bottom, right, top = rasterio.windows.bounds(
                window, 
                transform=image_file.transform
            )
            rw = rasterio.windows.from_bounds(
                left, bottom, right, top, 
                transform=raster_file.transform
            )
            image = raster_file.read(
                window=rw, 
                out_dtype=np.float32
            )
    return image

In [43]:
from rasterio.windows import Window
import numpy as np
window = Window(0, 0, 10000, 10000)
path = p+f'/Biarritz_EPSG32630_9.tif'
big_path = p+'/BIARRITZ_20140902_T_TOA_reproj-EPSG:32630.tif'
img = read_window_from_big_raster(window, path, big_path)

In [44]:
def read_window_basic(window, path):
    with rasterio.open(path) as image_file:
        image = image_file.read(window=window, out_dtype=np.float32)
    return image

In [50]:
window = Window(0, 0, 10000, 10000)
img2 = read_window_basic(window,big_path)

In [51]:
img2.shape

(4, 10000, 10000)

In [1]:
import rasterio
p = '/work/OT/ai4usr/fournip/digitanie/Toulouse/toulouse_full_tiled.tif'
ds = rasterio.open(p)
print(ds)

<open DatasetReader name='/work/OT/ai4usr/fournip/digitanie/Toulouse/toulouse_full_tiled.tif' mode='r'>


In [2]:
a = ds.statistics(bidx=1)

In [3]:
a

Statistics(min=-0.0029133798088878393, max=1.5431257486343384, mean=0.09988372604258908, std=0.05581474629124038)

In [4]:
a.min

-0.0029133798088878393